In [ ]:
!pip install carla opencv-python numpy ultralytics paho-mqtt

In [1]:
import carla, cv2, json
import numpy as np
from ultralytics import YOLO
import numpy as np
import paho.mqtt.client as mqtt

ModuleNotFoundError: No module named 'carla'

In [9]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()
world = client.load_world('Town02')

settings = world.get_settings()
settings.synchronous_mode = True # Enables synchronous mode
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

traffic_manager = client.get_trafficmanager()
traffic_manager.set_synchronous_mode(True)

In [10]:
def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

In [11]:
model = YOLO("./best.pt")

In [12]:
MQTT_BROKER = "mqtt.eclipseprojects.io"
MQTT_PORT = 1883
MQTT_TOPIC = "vehicle/detections"

mqtt_client = mqtt.Client()
mqtt_client.connect(MQTT_BROKER, MQTT_PORT, 60)

def send_mqtt_message(data):
    global last_message
    try:
        if data != last_message:
            payload = json.dumps(data)
            mqtt_client.publish(MQTT_TOPIC, payload)
            last_message = data
            print(f"Sent Message: {data}")
    except Exception as e:
        print(f"Error for MQTT: {e}")

c:\anaconda3\envs\carla-env\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  """


In [13]:

vehicle = spawn_vehicle()
world.tick()
camera = spawn_camera(attach_to=vehicle)

video_output = np.zeros((600, 800, 3), dtype=np.uint8)

last_message = None

def camera_callback(image):
    global last_message

    img_array = np.frombuffer(image.raw_data, dtype=np.uint8)
    img_array = img_array.reshape((image.height, image.width, 4))
    img_array = img_array[:, :, :3]
    results = model.predict(img_array, conf=0.6)

    output_image = results[0].plot()
    video_output[:] = output_image

    detections = [] 

    for box in results[0].boxes:
        label = model.names.get(box.cls.item())
        detection = {"label": label}
        detections.append(detection)

    if detections:
        for detection in detections:
            send_mqtt_message(detection)
    else:
        no_detection_message = {"label": "No Detection"}
        send_mqtt_message(no_detection_message)

camera.listen(lambda image: camera_callback(image))

vehicle.set_autopilot(True)

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        cv2.imshow('RGB Camera', video_output)
        world.tick()

finally:
    cv2.destroyAllWindows()
    camera.destroy()
    vehicle.destroy()
    mqtt_client.disconnect()



0: 480x640 (no detections), 100.4ms
Speed: 8.5ms preprocess, 100.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 96.2ms
Speed: 7.0ms preprocess, 96.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)



Sent Message: {'label': 'No Detection'}


0: 480x640 (no detections), 117.0ms
Speed: 8.5ms preprocess, 117.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 116.7ms
Speed: 7.2ms preprocess, 116.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 110.7ms
Speed: 7.0ms preprocess, 110.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 114.6ms
Speed: 7.0ms preprocess, 114.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 111.4ms
Speed: 7.0ms preprocess, 111.4ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 110.7ms
Speed: 8.5ms preprocess, 110.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 106.3ms
Speed: 10.0ms preprocess, 106.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 121.3ms
Speed: 6.5ms prepr

Sent Message: {'label': 'Red Light'}


0: 480x640 1 Red Light, 115.8ms
Speed: 10.5ms preprocess, 115.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 111.7ms
Speed: 8.5ms preprocess, 111.7ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 112.8ms
Speed: 9.5ms preprocess, 112.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)



...

Sent Message: {'label': 'No Detection'}
Sent Message: {'label': 'Green Light'}


0: 480x640 1 Green Light, 113.0ms
Speed: 5.0ms preprocess, 113.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 114.8ms
Speed: 7.0ms preprocess, 114.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 118.8ms
Speed: 9.0ms preprocess, 118.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)



Sent Message: {'label': 'No Detection'}


0: 480x640 (no detections), 110.4ms
Speed: 8.0ms preprocess, 110.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 138.3ms
Speed: 8.6ms preprocess, 138.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 109.9ms
Speed: 10.0ms preprocess, 109.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 112.2ms
Speed: 5.0ms preprocess, 112.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 114.4ms
Speed: 8.0ms preprocess, 114.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 118.3ms
Speed: 6.5ms preprocess, 118.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 117.8ms
Speed: 9.5ms preprocess, 117.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 111.2ms
Speed: 6.5ms prepr

0: 480x640 (no detections), 114.9ms
Speed: 8.5ms preprocess, 114.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
